# Making Structural Estimates From Empirical Results

This notebook conducts a quick and dirty structural estimation based on Table 9 of "MPC Heterogeneity and Household Balance Sheets" by Fagereng, Holm, and Natvik <cite data-cite="6202365/SUE56C4B"></cite>, who use Norweigian administrative data on income, household assets, and lottery winnings to examine the MPC from transitory income shocks (lottery prizes).  Their Table 9 reports an estimated MPC broken down by quartiles of bank deposits and
prize size; this table is reproduced here as $\texttt{MPC_target_base}$.  In this demo, we use the Table 9 estimates as targets in a simple structural estimation, seeking to minimize the sum of squared differences between simulated and estimated MPCs by changing the (uniform) distribution of discount factors.  The essential question is how well their results be rationalized by a simple one-asset consumption-saving model.  


The function that estimates discount factors includes several options for estimating different specifications:

1. TypeCount : Integer number of discount factors in discrete distribution; can be set to 1 to turn off _ex ante_ heterogeneity (and to discover that the model has no chance to fit the data well without such heterogeneity).
2. AdjFactor : Scaling factor for the target MPCs; user can try to fit estimated MPCs scaled down by (e.g.) 50%.
3. T_kill    : Maximum number of years the (perpetually young) agents are allowed to live.  Because this is quick and dirty, it's also the number of periods to simulate.
4. Splurge   : Amount of lottery prize that an individual will automatically spend in a moment of excitement (perhaps ancient tradition in Norway requires a big party when you win the lottery), before beginning to behave according to the optimal consumption function.  The patterns in Table 9 can be fit much better when this is set around \$700 --> 0.7.  That doesn't seem like an unreasonable amount of money to spend on a memorable party.
5. do_secant : Boolean indicator for whether to use "secant MPC", which is average MPC over the range of the prize.  MNW believes authors' regressions are estimating this rather than point MPC.  When False, structural estimation uses point MPC after receiving prize.  NB: This is incompatible with Splurge > 0.
6. drop_corner : Boolean for whether to include target MPC in the top left corner, which is greater than 1.  Authors discuss reasons why the MPC from a transitory shock *could* exceed 1.  Option is included here because this target tends to push the estimate around a bit.

In [194]:
# Import python tools

import sys
import os

import numpy as np
from copy import deepcopy

In [195]:
# Import needed tools from HARK

from HARK.utilities import approxUniform, getPercentiles
from HARK.parallel import multiThreadCommands
from HARK.estimation import minimizeNelderMead
from HARK.ConsumptionSaving.ConsIndShockModel import *
from HARK.cstwMPC.SetupParamsCSTW import init_infinite

In [196]:
# Set key problem-specific parameters

TypeCount = 8    # Number of consumer types with heterogeneous discount factors
AdjFactor = 1.0  # Factor by which to scale all of MPCs in Table 9
T_kill = 100     # Don't let agents live past this age
Splurge = 0.75    # Consumers automatically spend this amount of any lottery prize
do_secant = True # If True, calculate MPC by secant, else point MPC
drop_corner = 0.0 # If True, ignore upper left corner when calculating distance

In [197]:
# Set standard HARK parameter values

base_params = deepcopy(init_infinite)
base_params['LivPrb'] = [0.975]
base_params['Rfree'] = 1.04/base_params['LivPrb'][0]
base_params['PermShkStd'] = [0.1]
base_params['TranShkStd'] = [0.1]
base_params['T_age'] = T_kill # Kill off agents if they manage to achieve T_kill working years
base_params['AgentCount'] = 10000
base_params['pLvlInitMean'] = np.log(23.72) # From Table 1, in thousands of USD
base_params['T_sim'] = T_kill  # No point simulating past when agents would be killed off

In [198]:
# Define the MPC targets from Fagereng et al Table 9; element i,j is lottery quartile i, deposit quartile j

MPC_target_base = np.array([[1.047, 0.745, 0.720, 0.490],
                            [0.762, 0.640, 0.559, 0.437],
                            [0.663, 0.546, 0.390, 0.386],
                            [0.354, 0.325, 0.242, 0.216]])
MPC_target = AdjFactor*MPC_target_base

In [199]:
# Define the four lottery sizes, in thousands of USD; these are eyeballed centers/averages

lottery_size = np.array([1.625, 3.3741, 7.129, 40.0])

In [200]:
# Make several consumer types to be used during estimation

BaseType = IndShockConsumerType(**base_params)
EstTypeList = []
for j in range(TypeCount):
    EstTypeList.append(deepcopy(BaseType))
    EstTypeList[-1](seed = j)

In [201]:
# Define the objective function

def FagerengObjFunc(center,spread,verbose=False):
    '''
    Objective function for the quick and dirty structural estimation to fit
    Fagereng, Holm, and Natvik's Table 9 results with a basic infinite horizon
    consumption-saving model (with permanent and transitory income shocks).

    Parameters
    ----------
    center : float
        Center of the uniform distribution of discount factors.
    spread : float
        Width of the uniform distribution of discount factors.
    verbose : bool
        When True, print to screen MPC table for these parameters.  When False,
        print (center, spread, distance).

    Returns
    -------
    distance : float
        Euclidean distance between simulated MPCs and (adjusted) Table 9 MPCs.
    '''
    # Give our consumer types the requested discount factor distribution
    beta_set = approxUniform(N=TypeCount,bot=center-spread,top=center+spread)[1]
    for j in range(TypeCount):
        EstTypeList[j](DiscFac = beta_set[j])

    # Solve and simulate all consumer types, then gather their wealth levels
    multiThreadCommands(EstTypeList,['solve()','initializeSim()','simulate()','unpackcFunc()'])
    WealthNow = np.concatenate([ThisType.aLvlNow for ThisType in EstTypeList])

    # Get wealth quartile cutoffs and distribute them to each consumer type
    quartile_cuts = getPercentiles(WealthNow,percentiles=[0.25,0.50,0.75])
    for ThisType in EstTypeList:
        WealthQ = np.zeros(ThisType.AgentCount,dtype=int)
        for n in range(3):
            WealthQ[ThisType.aLvlNow > quartile_cuts[n]] += 1
        ThisType(WealthQ = WealthQ)

    # Keep track of MPC sets in lists of lists of arrays
    MPC_set_list = [ [[],[],[],[]],
                     [[],[],[],[]],
                     [[],[],[],[]],
                     [[],[],[],[]] ]

    # Calculate the MPC for each of the four lottery sizes for all agents
    for ThisType in EstTypeList:
        ThisType.simulate(1)
        c_base = ThisType.cNrmNow
        MPC_this_type = np.zeros((ThisType.AgentCount,4))
        
        for k in range(4): # Get MPC for all agents of this type
            Llvl = lottery_size[k]
            Lnrm = Llvl/ThisType.pLvlNow
            if do_secant:
                SplurgeNrm = Splurge/ThisType.pLvlNow
                mAdj = ThisType.mNrmNow + Lnrm - SplurgeNrm
                cAdj = ThisType.cFunc[0](mAdj) + SplurgeNrm
                MPC_this_type[:,k] = (cAdj - c_base)/Lnrm
            else:
                mAdj = ThisType.mNrmNow + Lnrm
                MPC_this_type[:,k] = cAdj = ThisType.cFunc[0].derivative(mAdj)

        # Sort the MPCs into the proper MPC sets
        for q in range(4):
            these = ThisType.WealthQ == q
            for k in range(4):
                MPC_set_list[k][q].append(MPC_this_type[these,k])

    # Calculate average within each MPC set
    simulated_MPC_means = np.zeros((4,4))
    for k in range(4):
        for q in range(4):
            MPC_array = np.concatenate(MPC_set_list[k][q])
            simulated_MPC_means[k,q] = np.mean(MPC_array)

    # Calculate Euclidean distance between simulated MPC averages and Table 9 targets
    diff = simulated_MPC_means - MPC_target
    if drop_corner:
        diff[0,0] = 0.0
    distance = np.sqrt(np.sum((diff)**2))
    if verbose:
        print(simulated_MPC_means)
    else:
        print (center, spread, distance)
    return distance

In [202]:
# Conduct the estimation

guess = [0.92,0.03]
f_temp = lambda x : FagerengObjFunc(x[0],x[1])
opt_params = minimizeNelderMead(f_temp, guess, verbose=True)
print('Finished estimating for scaling factor of ' + str(AdjFactor) + ' and "splurge amount" of $' + str(1000*Splurge))
print('Optimal (beta,nabla) is ' + str(opt_params) + ', simulated MPCs are:')
dist = FagerengObjFunc(opt_params[0],opt_params[1],True)
print('Distance from Fagereng et al Table 9 is ' + str(dist))

0.92 0.03 0.5728359985856661
0.9660000000000001 0.03 1.0865233968407848
0.92 0.0315 0.5707029208693234
0.874 0.0315 0.5023837137685602
0.8280000000000001 0.03225 0.6060752772205258
0.874 0.033 0.4997399788082691
0.8509999999999998 0.0345 0.5434101687543998
0.828 0.033 0.6052681667980814
0.897 0.031875 0.4914268289260696
0.8969999999999999 0.033375 0.4886712751120285
0.9084999999999999 0.034312499999999996 0.5119247641785709
0.92 0.03225 0.5696710469907433
0.8855 0.0328125 0.48776600094040784
0.8854999999999997 0.03431250000000001 0.4849927681594527
0.8797499999999996 0.035531250000000014 0.4875623664057135
0.8739999999999998 0.033750000000000016 0.4985091271811477
0.8912499999999999 0.033468750000000005 0.4851906123183567
0.8912499999999997 0.034968750000000014 0.4823916520349069
0.8941249999999994 0.03604687500000002 0.48151546386843297
0.8883749999999992 0.036890625000000024 0.47889436229502247
0.8869374999999988 0.03860156250000003 0.47591510716208757
0.8955624999999985 0.0403359375

### PROBLEM

See what happens if you do not allow a splurge amount at all.  Hint: Think about how this question relates to the `drop_corner` option.

Explain why you get the results you do, and comment on possible interpretations of the "splurge" that might be consistent with economic theory.    
Hint: What the authors are able to measure is actually the marginal propensity to EXPEND, not the marginal propensity to CONSUME as it is defined in our benchmark model.

# Put your solution here

The authors estimates an average increase in consumption expenditure and build hypotese why it is consistent with how MPCs are extimated and interpreted in the literature. A point estimate of beta 1 will be pulled toward the MPCs of winner of relatevely high size. The authors estimates how large fraction of the lottery prize the average lottery winner in the sample spends on consumption within the year of winning. 
 
The "splurge" is interpreted here as a state when consumers automatically spend the amount of any lottery prize. 
Compare to the incomplete markets models where households face uninsurable idiosyncratic risk and a borrowing constraints, they acqure a buffer stock of capital in order to prevent the constraint from binding, Then the wealth level is the main determinant of the households' MPC then their net wealth level. However, the authors argues that their findings indicate that the net wealth is important once liquidity is controlled for. These findings are in conflict with the interpretation from the buffer stock savings models. 

"Drop corner" is the boolean for whether to include target MPC in the top left corner, which is greater than 1. MPC from the transitory shock could exceed 1. However in case if we do not allow a splurge amount at all as a shock size, then the response can differ but not so much. "Drop corner" option can push estimates and show what happened if variabel "splurge" is equal 0 or 1. Lets look at the 3 scenario: splurge=0,75 and drop corner=1; splurge=0,0 and drop corner=0 and splurge=0,75 and drop corner=0

Scenario 1
Splurge=0,0
drop corner=0


[[0.77361336 0.68317127 0.56461082 0.40476962]
 [0.74354975 0.66482752 0.55301552 0.39626053]
 [0.70353353 0.63512154 0.5305429  0.3793119 ]
 [0.5613238  0.50428804 0.4125933  0.29261249]]
 Distance from Fagereng et al Table 9 is 0.5021025392131131
 Optimal (beta,nabla) is [0.78981881 0.16098057]
 
When individuals spend nothing and drop corner is zero, the calculated distance from the target values is the biggest among all scenarios. It is looks like the worst scenario as it shows that if there is no splurge it is difficut for the model to match calculated high MPC to the lower wealth qualriles. 

Scenario 2
Splurge=0,75
drop corner=1
 
 [[0.77190491 0.74430021 0.70140645 0.59364668]
 [0.65701731 0.62152995 0.56087767 0.4084999 ]
 [0.57760029 0.54386887 0.47673001 0.31037197]
 [0.39990788 0.37986204 0.32617723 0.19521486]]
 Distance from Fagereng et al Table 9 is 0.23757458381351887
Optimal (beta,nabla) is [0.87162876 0.09774449]

In this scenario beta is the largest, nabla and distnace is the lowest. Probably the best scenario for data simulation. The model shows that if there are some expenditure and drop corner=1, then the distance between the simulated MPC and target MPC is low. It shows that relative high beta allows the model better to match simulated MPC to the target MPC . 
 
Scenario 3
Splurge=0,75
drop corner=0

[[0.78791058 0.75721641 0.70740854 0.58335306]
 [0.68018181 0.64008361 0.56962574 0.39541177]
 [0.60439296 0.5651436  0.48660528 0.29588357]
 [0.42933477 0.40248463 0.33554083 0.18337192]]
Distance from Fagereng et al Table 9 is 0.3569805410320015
Optimal (beta,nabla) is [0.86275795 0.11378181]

This scenario is the nest best. The beta is some lower than in the previous scenario when the drop corner=1. Now it we use the drop corner=0, then high beta also allows to match simulated MPC to the target MPC when simulated MPC is some lower than in the previous scenario. But this scenario shows that if we exclude the drop corner the model might work better to match relative high simulated MPC to their target level.  

### PROBLEM

Call the _Marginal Propensity to Continue Consuming_ (MPCC) in year `t+n` the proportion of lottery winnings that get spent in year `t+n`.  That is, if consumption is higher in year `t+2` by an amount corresponding to 14 percent of lottery winnings, we would say  _the MPCC in t+2 is 14 percent.

For the baseline version of the model with the "splurge" component, calculate the MPCC's for years `t+1` through `t+3` and plot them together with the MPC in the first year (including the splurge component)


In [222]:
# Put your solution here
def FagerengFutureObjFunc(center,spread,verbose=False):

    # Give our consumer types the requested discount factor distribution
    beta_set = approxUniform(N=TypeCount,bot=center-spread,top=center+spread)[1]
    for j in range(TypeCount):
        EstTypeList[j](DiscFac = beta_set[j])
        # add tracking vars to each Type
        EstTypeList[j].track_vars = ['aNrmNow','cNrmNow','mNrmNow','pLvlNow','PermShkNow','TranShkNow']
        
    #aNrmNow - A 1D array of end-of-period assets; 
    #cNrmNow  – Arrays with consumption points for interpolation now;
    #mNrmNow  – Arrays with corresponding market resource points for interpolation now;
    #pLvlNow - Arrays with permanent income levels, listed by each ConsumerType in self.agents.
    #PermShkNow - Arrays with permanent income shocks, listed by each ConsumerType in self.agents.
    #TranShkNow - Arrays with transitory income shocks, listed by each ConsumerType in self.agents.
     
      # Bounds for axis with size 100
    Start_Period = 95;
    # Solve and simulate all consumer types, then gather their wealth levels
    multiThreadCommands(EstTypeList,['solve()','initializeSim()','simulate(95)','unpackcFunc()'])
    WealthNow = np.concatenate([ThisType.aLvlNow for ThisType in EstTypeList])
    Rfree = base_params['Rfree']

    # Get wealth quartile cutoffs and distribute them to each consumer type
    quartile_cuts = getPercentiles(WealthNow,percentiles=[0.25,0.50,0.75])
    for ThisType in EstTypeList:
        WealthQ = np.zeros(ThisType.AgentCount,dtype=int)
        for n in range(3):
            WealthQ[ThisType.aLvlNow > quartile_cuts[n]] += 1
        ThisType(WealthQ = WealthQ)

    # Keep track of MPC sets in lists of lists of arrays
    MPC_set_list = [ [[],[],[],[]],
                     [[],[],[],[]],
                     [[],[],[],[]],
                     [[],[],[],[]] ]
    MPC_set_list_t1 = deepcopy(MPC_set_list)
    MPC_set_list_t2 = deepcopy(MPC_set_list)
    MPC_set_list_t3 = deepcopy(MPC_set_list)

   
    # Calculate the MPC for each of the four lottery sizes for all agents
    
    #for ThisType in EstTypeList:
       # ThisType.simulate(1)
        #c_base = ThisType.cNrmNow
        #MPC_this_type = np.zeros((ThisType.AgentCount,4))
        
    for ThisType in EstTypeList:
        ThisType.simulate(4)
        c_base_0 = ThisType.cNrmNow_hist[Start_Period]
        c_base_t1 = ThisType.cNrmNow_hist[Start_Period+1]
        c_base_t2 = ThisType.cNrmNow_hist[Start_Period+2]
        c_base_t3 = ThisType.cNrmNow_hist[Start_Period+3]
        
        MPC_this_type = np.zeros((ThisType.AgentCount,4))
        MPC_this_type_t1 = np.zeros((ThisType.AgentCount,4))
        MPC_this_type_t2 = np.zeros((ThisType.AgentCount,4))
        MPC_this_type_t3 = np.zeros((ThisType.AgentCount,4))
    
        for k in range(4): # Get MPC for all agents of this type       
            Llvl = lottery_size[k]
            Lnrm = Llvl/ThisType.pLvlNow_hist[Start_Period]     
            SplurgeNrm = Splurge/ThisType.pLvlNow_hist[Start_Period]
            mAdj = ThisType.mNrmNow_hist[Start_Period] + Lnrm - SplurgeNrm
            cAdj = ThisType.cFunc[0](mAdj) + SplurgeNrm
            MPC_this_type[:,k] = (cAdj - c_base_0) /Lnrm
            
            # MPC for t+1 
            Llvl = lottery_size[k]
            Lnrm = Llvl/ThisType.pLvlNow_hist[Start_Period+1]
            aNrm_t0 = ThisType.mNrmNow_hist[Start_Period] + Lnrm - cAdj
            aLvl_t0 = aNrm_t0*ThisType.pLvlNow_hist[Start_Period]
            mLvl_t1 = aLvl_t0*Rfree + ThisType.TranShkNow_hist[Start_Period+1]*ThisType.pLvlNow_hist[Start_Period+1]
            mNrm_t1 = mLvl_t1/ThisType.pLvlNow_hist[Start_Period+1]
            cNrm_t1 = ThisType.cFunc[0](mNrm_t1)
            MPC_this_type_t1[:,k] = (cNrm_t1 - c_base_t1) /Lnrm

            # MPC for t+2 
            Llvl = lottery_size[k]
            Lnrm = Llvl/ThisType.pLvlNow_hist[Start_Period+2]
            aNrm_t1 = mNrm_t1 - cNrm_t1;
            aLvl_t1 = aNrm_t1*ThisType.pLvlNow_hist[Start_Period+1]
            mLvl_t2 = aLvl_t1*Rfree + ThisType.TranShkNow_hist[Start_Period+2]*ThisType.pLvlNow_hist[Start_Period+2]
            mNrm_t2 = mLvl_t2/ThisType.pLvlNow_hist[Start_Period+2]
            cNrm_t2 = ThisType.cFunc[0](mNrm_t2)
            MPC_this_type_t2[:,k] = (cNrm_t2 - c_base_t2) /Lnrm

            # MPC for t+3 
            Llvl = lottery_size[k]
            Lnrm = Llvl/ThisType.pLvlNow_hist[Start_Period+3]
            aNrm_t2 = mNrm_t2 - cNrm_t2;
            aLvl_t2 = aNrm_t2*ThisType.pLvlNow_hist[Start_Period+2]
            mLvl_t3 = aLvl_t2*Rfree + ThisType.TranShkNow_hist[Start_Period+3]*ThisType.pLvlNow_hist[Start_Period+3]
            mNrm_t3 = mLvl_t3/ThisType.pLvlNow_hist[Start_Period+3]
            cNrm_t3 = ThisType.cFunc[0](mNrm_t3)
            MPC_this_type_t3[:,k] = (cNrm_t3 - c_base_t3) /Lnrm
            

        # Sort the MPCs into the proper MPC sets
        for q in range(4):
            these = ThisType.WealthQ == q
            for k in range(4):
                MPC_set_list[k][q].append(MPC_this_type[these,k])
                MPC_set_list_t1[k][q].append(MPC_this_type_t1[these,k])
                MPC_set_list_t2[k][q].append(MPC_this_type_t2[these,k])
                MPC_set_list_t3[k][q].append(MPC_this_type_t3[these,k])

    # Calculate average within each MPC set
    simulated_MPC_means = np.zeros((4,4))
    simulated_MPC_means_t1 = np.zeros((4,4))
    simulated_MPC_means_t2 = np.zeros((4,4))
    simulated_MPC_means_t3 = np.zeros((4,4))
    for k in range(4):
        for q in range(4):
            MPC_array = np.concatenate(MPC_set_list[k][q])
            simulated_MPC_means[k,q] = np.mean(MPC_array)
            simulated_MPC_means_t1[k,q] = np.mean(np.concatenate(MPC_set_list_t1[k][q]))
            simulated_MPC_means_t2[k,q] = np.mean(np.concatenate(MPC_set_list_t2[k][q]))
            simulated_MPC_means_t3[k,q] = np.mean(np.concatenate(MPC_set_list_t3[k][q]))
            
    print('The MPC for t+0 \n', simulated_MPC_means)
    print('\n')
    print('The MPCC for t+1 \n', simulated_MPC_means_t1)
    print('\n')
    print('The MPCC for t+2 \n', simulated_MPC_means_t2)
    print('\n')
    print('The MPCC for t+3 \n', simulated_MPC_means_t3)
    print('\n')
   
    # Calculate Euclidean distance between simulated MPC averages and Table 9 targets
    diff = simulated_MPC_means - MPC_target
    if drop_corner:
        diff[0,0] = 0.0
    distance = np.sqrt(np.sum((diff)**2))
    return distance

dist = FagerengFutureObjFunc(opt_params[0],opt_params[1],True)

The MPC for t+0 
 [[0.78567857 0.75529075 0.70422096 0.57434956]
 [0.67596163 0.63714129 0.56530426 0.38258846]
 [0.59784702 0.56132056 0.48191713 0.28327884]
 [0.42130211 0.39851849 0.33303836 0.17886577]]


The MPCC for t+1 
 [[0.15017068 0.17156936 0.19343695 0.20817884]
 [0.20508986 0.21869224 0.22489322 0.18006798]
 [0.23801211 0.24456214 0.24006372 0.16486564]
 [0.26434538 0.2607976  0.23505038 0.14107942]]


The MPCC for t+2 
 [[0.08420981 0.11176419 0.16050027 0.28118279]
 [0.10095318 0.12195854 0.15508628 0.19716259]
 [0.11762377 0.13275262 0.15614253 0.15623452]
 [0.16672645 0.17023348 0.17019776 0.1233315 ]]


The MPCC for t+3 
 [[0.05853968 0.08205659 0.13928964 0.34976285]
 [0.05648862 0.07443471 0.11433888 0.21729341]
 [0.06136254 0.07528211 0.106014   0.15338417]
 [0.10061274 0.10719368 0.12161918 0.10912477]]


